In [16]:
using JuMP, Gurobi, Random, Statistics, Combinatorics, LinearAlgebra
using DataFrames, CSV, IterTools
using Random
using GLMNet, StatsBase

seed = 2
gurobi_env = Gurobi.Env()
Random.seed!(seed)

df_path = "data/output/preprocessed.csv"
predictor_col = "income_total"
normalization_type = "std"


Academic license - for non-commercial use only - expires 2022-09-05


"std"

In [17]:
function calc_r2(X, y, beta)
    X = augment_X(X)
    SSres = sum( (y .- X*beta).^2 )
    SStot = sum( (y .- Statistics.mean(y)).^2 )
    return 1-SSres/SStot
end

function grid_search(X, y, solver_func, error_func, error_strategy="Min",train_val_ratio=0.7;params... )

    # Split the data into training/validation
    X_train, y_train, X_val, y_val = partitionTrainTest(X, y, train_val_ratio);
    
    # Create the grid (i.e. all the combinations of the given parameters)
    param_names = keys(params)
    param_combinations = [
        Dict(param_names[i]=>p[i] for i in 1:length(param_names)) 
        for p in product([params[i] for i in keys(params)]...)
    ]
    
    # Initialize variables used to hold validation information
    error_multiplier = error_strategy == "Min" ? 1 : -1
    best_error = Inf # We consider minimization
    best_param_set = []
    
    # Iterate over all combinations of parameters
    for param_comb in param_combinations
        
        # Optimize model and find optimal variables
        model_vars = solver_func(X_train,y_train;param_comb...)
        
        # Evaluate model error on validation set
        if model_vars isa Tuple
            error = error_multiplier*error_func(X_val, y_val, model_vars...)
        else
            error = error_multiplier*error_func(X_val, y_val, model_vars)
        end
        
        # If error is better than the best error so far, keep track 
        # of the error and the params
        if error < best_error
            best_error = error 
            best_param_set = param_comb
        end
    end
    
    # Retrain the model on the whole training set 
    # using the best set of params
    model_vars = solver_func(X,y;best_param_set...)
    
    # Return the model variable and the best params
    return model_vars, best_param_set
end

function normalize_data(X, method="minmax"; is_train=true)
    X = copy(X)
    if is_train
        global nonzero_idx = findall([maximum(X[:,i])-minimum(X[:,i]) for i = 1:size(X,2)].>=0.01)
        if method == "std"
            global dt=fit(ZScoreTransform, X[:,nonzero_idx]; dims=1, center=true, scale=true)
        elseif method == "minmax"
            global dt=fit(UnitRangeTransform, X[:,nonzero_idx]; dims=1, unit=true)
        end
    end
    X[:,nonzero_idx] = StatsBase.transform(dt, X[:,nonzero_idx])
    
    return X
end


function partitionTrainTest(X,y, at = 0.7, s=seed)
    n = size(X,1)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    return X[train_idx,:], y[train_idx], X[test_idx,:], y[test_idx]
end

function augment_X(X)
    return [X ones(size(X,1),1)]
end

function solve_holistic_regr(X,y;gamma,rho,k)
    C = cor(X)
    n,p = size(X)
    X_aug = augment_X(X)
    M = 10^5
    m = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    set_optimizer_attribute(m, "OutputFlag", 0)
    set_optimizer_attribute(m, "PSDTol", 1)
    @variable(m, beta[1:(p+1)])
    @variable(m, z[1:p],Bin)
    @variable(m, t[1:p])
    @objective(m, Min, 1/2*sum((X_aug*beta.-y).^2)+gamma*sum(t[i] for i=1:p))
    @constraint(m, [i=1:p], t[i]>= beta[i])
    @constraint(m, [i=1:p], t[i]>= -beta[i])
    @constraint(m, [i=1:p], beta[i]<= M*z[i])
    @constraint(m, [i=1:p], -M*z[i]<=beta[i])
    @constraint(m, sum(z)<=k)
    #@constraint(m, [i=1:4:p-3], sum(z[i+j] for j=0:3)<=1)
#     for i in 1:p
#         for j in i+1:p
#             if abs(C[i,j]) > rho
#                 @constraint(m, z[i]+z[j] <= 1)
#             end
#         end
#     end
    optimize!(m)
    return JuMP.value.(beta)
end

function fit_lasso(X, y)
    cv = glmnetcv(X, y);
    id_best = argmin(cv.meanloss);
    betas = [GLMNet.coef(cv);cv.path.a0[id_best]];
    return betas
end

fit_lasso (generic function with 1 method)

In [20]:
df = DataFrame(CSV.File(df_path, header=1))
df = last(df, 100000)
names(df)

271-element Vector{String}:
 "age"
 "cost_fuel"
 "cost_gas"
 "earnings_total"
 "electricity_cost"
 "family_people_number"
 "gross_rent"
 "gross_rent_pcnt_income"
 "household_people_number"
 "income_adjustment_factor"
 "income_all"
 "income_family"
 "income_household"
 ⋮
 "field_of_degree_54"
 "field_of_degree_55"
 "field_of_degree_56"
 "field_of_degree_57"
 "field_of_degree_59"
 "field_of_degree_60"
 "field_of_degree_61"
 "field_of_degree_62"
 "num_degrees_0.0"
 "num_degrees_1.0"
 "num_degrees_2.0"
 "num_degrees_nan"

In [21]:
excluded_cols = [
    "earnings_total",
    "income_interest_dividends_rental",
    "income_retirement",
    "income_all",
    "income_social_security",
    "income_supplementary_security",
    "income_total",
    "income_self_employment",
    "income_household",
    "income_to_poverty_ratio",
    "income_public_assistance",
    "income_family",
    "income_wages_salary",
    "monthly_owner_costs",
    "gross_rent",
    "person_number",
    "rent_monthly",
    "property_value"
]
cols = filter(x -> x ∉ excluded_cols, names(df))
X, y = Matrix{Float32}(df[!, filter(x -> x != predictor_col, cols)]), df[!,predictor_col]

X_train, y_train, X_test, y_test = partitionTrainTest(X, y, 0.7);
X_train = normalize_data(X_train, normalization_type; is_train=true);
X_test = normalize_data(X_test, normalization_type; is_train=false);

In [22]:
#betas, params = grid_search(X_train, y_train, solve_holistic_regr, calc_r2, "Max", 0.7; gamma=[0.1], rho=[0.7], k=[20])
betas = fit_lasso(X_train, y_train)
r2_c = calc_r2(X_test, y_test, betas)


0.3867315577314535

In [23]:
#cols = filter(x -> x ∉ excluded_cols, names(df))
#important_features = cols[findall(abs.(betas_hol) .>= 0.01)]
THRESHOLD = 0.000001
println("Most important features:")
for i in sortperm(abs.(betas[2:end]), rev=true)
    if abs(betas[i])<=THRESHOLD
        continue
    end
    println("- $(cols[i]) : $(betas[i+1])")
end


Most important features:
- own_children_number : 14076.32361325964
- field_of_degree_1_64.0 : -7962.0311873300425
- family_people_number : -7946.227509730238
- meals_in_rent_nan : 6751.734905287788
- own_children_presence_4.0 : 6589.94808360631
- meals_in_rent_1.0 : 6442.4207590645165
- occupation_code_3.0 : 4107.206785778046
- occupation_code_7.0 : 4015.754991638387
- field_of_degree_23 : 3872.6161294715384
- own_children_presence_1.0 : -3851.5884531724128
- educational_attainment_5.0 : 3728.8307133362814
- english_language_nan : -3691.8366352440485
- mortgage_second_payment : 3361.7281800230708
- gross_rent_pcnt_income : -3303.841628193352
- field_of_degree_20 : 2933.7209827741135
- occupation_code_26.0 : 2807.5485980509006
- meals_in_rent_2.0 : -2759.571863371828
- family_employment_status_1.0 : 2735.9069843288967
- worker_class_3.0 : -2650.372605909005
- occupation_code_2.0 : 2563.566817748766
- worker_class_2.0 : -2475.822998106382
- worker_class_6.0 : 2339.198239433422
- occupati